In [7]:
import os
import sys
import glob
import time
import random
import pandas as pd
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from lib.config import *
from lib.encoder.ffn import FFNEncoder, FFNDecoder
from lib.encoder.vqvae import VQVAE, VectorQuantizer
from lib.train.run_autoencoder_training import AutoencoderTrainer
from lib.utils.pose import get_pose_estimation

In [3]:
mp4_files = glob.glob("dataset/corpus/*.mp4")
mp4_files

['dataset/corpus/GEREK _1.mp4',
 'dataset/corpus/I╠çLGINC╠ğ_0.mp4',
 'dataset/corpus/MEMUR_1.mp4',
 'dataset/corpus/BAS╠ğPARMAK_0.mp4',
 'dataset/corpus/YILDIZ_0.mp4',
 'dataset/corpus/ZARF_0.mp4',
 'dataset/corpus/SERVIS_1.mp4',
 'dataset/corpus/HAKLI_0.mp4',
 'dataset/corpus/KIM_1.mp4',
 'dataset/corpus/I╠çKISI_0.mp4',
 'dataset/corpus/DIG╠åER_0.mp4',
 'dataset/corpus/BAYRAK_0.mp4',
 'dataset/corpus/I╠çNS╠ğALLAH_1.mp4',
 'dataset/corpus/HOPARLO╠êR_0.mp4',
 'dataset/corpus/I╠çMZALAMAK_0.mp4',
 'dataset/corpus/GARANTI_0.mp4',
 'dataset/corpus/KUSURA BAKMAMAK_1.mp4',
 'dataset/corpus/SAC╠ğ_0.mp4',
 'dataset/corpus/VERGI _1.mp4',
 'dataset/corpus/YAVAS╠ğLAMAK_0.mp4',
 'dataset/corpus/BAG╠åIMSIZ _0.mp4',
 'dataset/corpus/GO╠êNU╠êL_0.mp4',
 'dataset/corpus/TREN_1.mp4',
 'dataset/corpus/C╠ğAY_1.mp4',
 'dataset/corpus/UC╠ğAK_0.mp4',
 'dataset/corpus/ALET_0.mp4',
 'dataset/corpus/PRATIK_0.mp4',
 'dataset/corpus/I╠çC╠ğIN_0.mp4',
 'dataset/corpus/BIRA_1.mp4',
 'dataset/corpus/

In [9]:
analyze_results = []

for i, analyze_video in enumerate(mp4_files):
    print(f"Analyzing {i}/{len(mp4_files)} : {analyze_video}")

    estimation, info = get_pose_estimation(analyze_video, with_info=True)

    pd_info = pd.DataFrame(info, columns=["video_name", "info", "value"])

    num_of_frames_total = pd_info[pd_info['info']
                                  == 'total_number_of_frames']["value"].iloc[0]
    num_of_frames_missing_pose = 0
    num_of_frames_missing_left_hand = 0
    num_of_frames_missing_right_hand = 0

    if "missing_pose" in pd_info["info"].to_list():
        num_of_frames_missing_pose = pd_info.value_counts("info")[
            "missing_pose"]

    if "missing_left_hand" in pd_info["info"].to_list():
        num_of_frames_missing_left_hand = pd_info.value_counts("info")[
            "missing_left_hand"]

    if "missing_right_hand" in pd_info["info"].to_list():
        num_of_frames_missing_right_hand = pd_info.value_counts("info")[
            "missing_right_hand"]

    # print(f"Video:                              {analyze_video}")
    # print(f"# of total frames:                  {num_of_frames_total}")
    # print(f"# of missing frames for pose:       {num_of_frames_missing_pose}")
    # print(f"# of missing frames for left hand:  {num_of_frames_missing_left_hand}")
    # print(f"# of missing frames for right hand: {num_of_frames_missing_right_hand}")
    # print(GLOBAL_CONFIG)

    analyze_results.append([analyze_video,
                           num_of_frames_total,
                           num_of_frames_missing_pose,
                           num_of_frames_missing_left_hand,
                           num_of_frames_missing_right_hand,
                           GLOBAL_CONFIG])

Analyzing 0/3395 : dataset/corpus/GEREK _1.mp4


I0000 00:00:1711849654.765789       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


Analyzing 1/3395 : dataset/corpus/I╠çLGINC╠ğ_0.mp4


I0000 00:00:1711849656.851386       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


Analyzing 2/3395 : dataset/corpus/MEMUR_1.mp4


...

Analyzing 99/3395 : dataset/corpus/TAKIP ETMEK_0.mp4


I0000 00:00:1711849871.143466       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


Analyzing 100/3395 : dataset/corpus/KIRILMAK_0.mp4


I0000 00:00:1711849873.285028       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


KeyboardInterrupt: 

In [11]:
pd_analyze_results = pd.DataFrame(analyze_results,
                                  columns=["video_name", 
                                           "num_of_total_frames", 
                                           "num_of_missing_frames_pose",
                                           "num_of_missing_frames_left_hand",
                                           "num_of_missing_frames_right_hand",
                                           "configuration"
                                           ])
pd_analyze_results

,video_name,num_of_total_frames,num_of_missing_frames_pose,num_of_missing_frames_left_hand,num_of_missing_frames_right_hand,configuration
0,dataset/corpus/GEREK _1.mp4,51,0,51,10,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
1,dataset/corpus/I╠çLGINC╠ğ_0.mp4,56,0,56,29,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
2,dataset/corpus/MEMUR_1.mp4,60,0,60,33,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3,dataset/corpus/BAS╠ğPARMAK_0.mp4,41,0,41,19,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
4,dataset/corpus/YILDIZ_0.mp4,66,0,22,21,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
...,...,...,...,...,...,...
95,dataset/corpus/SIKINTI_0.mp4,40,0,40,11,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
96,dataset/corpus/BALIK_1.mp4,71,0,41,41,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
97,dataset/corpus/C╠ğEKILMEK_1.mp4,44,0,12,14,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
98,dataset/corpus/BU╠êYU╠êTMEK _0.mp4,77,0,45,43,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."


In [14]:
# Seperator intentionally given as semicolon ";"
# as configuration has comma ","

import datetime
now = datetime.datetime.now()
formatted_datetime = now.strftime("%Y%m%d_%H%M%S")

pd_analyze_results.to_csv(f"analyze/{formatted_datetime}.csv" ,sep=';')